## Home exercise below
a) Build a language model based on n-grams using the Laplace smoothing method for the following models:
- 1-gram
- 2-gram
- 3-gram

Import necessary libraries: nltk, gdown,...

In [47]:
import nltk
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, TreebankWordTokenizer
from collections import Counter, defaultdict
import re
import math
import gdown
nltk.download('punkt')  


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Use gdown to download tedtalk.txt from the google drive link.

In [48]:
url=f"https://drive.google.com/uc?export=download&id=1tQ9zW0ihL3Uc6GIge9AIV1pQnHYI7ihI"
output_path="tedtalk.txt"
gdown.download(url,output_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1tQ9zW0ihL3Uc6GIge9AIV1pQnHYI7ihI
To: c:\HCMUT\Study\HK242\NLP\Exercise\Lab3\Home Exercise\tedtalk.txt
100%|██████████| 40.3M/40.3M [00:01<00:00, 37.6MB/s]


'tedtalk.txt'

Open the plain text file and read into `corpus`.

In [49]:
# Load text corpus from file
with open('tedtalk.txt', 'r', encoding='utf-8') as file:
    corpus = file.read()


Preprocess the text by lowering, and keeping only `[a-z0-9 .!?\']`.
Then use `nltk.sent_tokenize()` to tokenize text into list of sentences and tokenize each sentence into list of words using `nltk.tokenize()`.

In [50]:
# Preprocess text: lowercase, remove punctuation, tokenize
def preprocess(text):
    text = text.lower()
    cleaned_text = re.sub(r"[^a-z0-9 .!?\']", " ", text)
    return cleaned_text


def split_sentences_and_tokenize(text):
    tokenizer = TreebankWordTokenizer()
    sentences = sent_tokenize(text)
    tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    
    return tokenized_sentences


Function to generate ngrams from sentences. Add `<s>` and `</s>` for 2-gram and 3-gram.

In [51]:
# Generate n-grams
def generate_ngrams_from_sentences(sentences, n):
    ngrams_list = []
    for sentence in sentences:
        padded_sentence=['<s>']+sentence #To efficiently calculate prefix
        if n>1:
            padded_sentence = ['<s>'] * (n - 1) + padded_sentence + ['</s>'] 
        ngrams_list.extend(list(ngrams(padded_sentence, n)))
    return ngrams_list


Function to precompute ngram probablity to optimize runtime.

In [52]:
# Function to compute n-gram probabilities with Laplace smoothing
def compute_ngram_probs(vocabulary_size,token_size, n,ngram_counts, alpha=1):
    probs = {}
    
    for ngram, count in ngram_counts[n-1].items():
        if n == 1:  # For unigrams, no need for prefix_count
            prefix_count = token_size 
        else:  # For bigrams or trigrams, use the prefix
            prefix = ngram[:-1]  
            prefix_count = ngram_counts[n-2].get(prefix, 0)
        # Apply Laplace smoothing
        probs[ngram] = (count + alpha) / (prefix_count + alpha * vocabulary_size)
    
    return probs
    

Preprocess and tokenize based on sentences.

In [53]:
sentences=preprocess(corpus)
sentences = split_sentences_and_tokenize(sentences)
print(sentences[:20])

[['thank', 'you', 'so', 'much', 'chris', '.'], ['and', 'it', "'s", 'truly', 'a', 'great', 'honor', 'to', 'have', 'the', 'opportunity', 'to', 'come', 'to', 'this', 'stage', 'twice', 'i', "'m", 'extremely', 'grateful', '.'], ['i', 'have', 'been', 'blown', 'away', 'by', 'this', 'conference', 'and', 'i', 'want', 'to', 'thank', 'all', 'of', 'you', 'for', 'the', 'many', 'nice', 'comments', 'about', 'what', 'i', 'had', 'to', 'say', 'the', 'other', 'night', '.'], ['and', 'i', 'say', 'that', 'sincerely', 'partly', 'because', 'mock', 'sob', 'i', 'need', 'that', '.'], ['laughter', 'put', 'yourselves', 'in', 'my', 'position', '.'], ['laughter', 'i', 'flew', 'on', 'air', 'force', 'two', 'for', 'eight', 'years', '.'], ['laughter', 'now', 'i', 'have', 'to', 'take', 'off', 'my', 'shoes', 'or', 'boots', 'to', 'get', 'on', 'an', 'airplane', '!'], ['laughter', 'applause', 'i', "'ll", 'tell', 'you', 'one', 'quick', 'story', 'to', 'illustrate', 'what', 'that', "'s", 'been', 'like', 'for', 'me', '.'], ['lau

Calculate total number of tokens, calculate the vocabulary size, count unigram, bigram, trigram and use them to precompute the probabilities.

In [54]:
all_tokens = [token for sentence in sentences for token in sentence]
vocabulary_size = len(set(all_tokens))  
token_size=len(all_tokens)
print("Total tokens: ",token_size)
print("Vocabulary size: ", vocabulary_size)
ngram_counts=[]
ngram_probs=[]
for x in range(3):
    ngram_counts.append(Counter(generate_ngrams_from_sentences(sentences,x+1)))
for x in range(3):
    ngram_probs.append(compute_ngram_probs(vocabulary_size,token_size,x+1,ngram_counts))


Total tokens:  7835609
Vocabulary size:  69944


In [ ]:

print("Sample Unigram Probabilities:", list(ngram_probs[0].items())[1:15])
print("Sample Bigram Probabilities:", list(ngram_probs[1].items())[1:15])
print("Sample Trigram Probabilities:", list(ngram_probs[2].items())[1:15])


Sample Unigram Count: [(('<s>',), 441023), (('thank',), 5195), (('you',), 108403), (('so',), 57089), (('much',), 9220), (('chris',), 498), (('.',), 400768), (('and',), 240579), (('it',), 115807), (("'s",), 85344), (('truly',), 695), (('a',), 170846), (('great',), 4756), (('honor',), 277), (('to',), 210927), (('have',), 44958), (('the',), 336385), (('opportunity',), 1156), (('come',), 6146), (('this',), 74906)]
Sample Bigram Count: [(('<s>', '<s>'), 441023), (('<s>', 'thank'), 3222), (('thank', 'you'), 5001), (('you', 'so'), 356), (('so', 'much'), 1751), (('much', 'chris'), 4), (('chris', '.'), 56), (('.', '</s>'), 400542), (('<s>', 'and'), 60110), (('and', 'it'), 8960), (('it', "'s"), 32904), (("'s", 'truly'), 25), (('truly', 'a'), 20), (('a', 'great'), 1219), (('great', 'honor'), 17)]
Sample Trigram Count: [(('<s>', '<s>', '<s>'), 441023), (('<s>', '<s>', 'thank'), 3222), (('<s>', 'thank', 'you'), 3194), (('thank', 'you', 'so'), 259), (('you', 'so', 'much'), 261), (('so', 'much', 'chr

Function to calculate probability of a sentence, fall back to computing again using ngram counts if the precomputation is not found.

In [56]:
def calculate_sentence_probability(sentence, ngram_probs,ngram_count,token_size,vocabulary_size, n, alpha=1):
    tokens = [word for sentence in split_sentences_and_tokenize(preprocess(sentence)) for word in sentence]
    prob = 1.0

    if n > 1:
        tokens = ['<s>'] * (n - 1) + tokens + ['</s>']  # Add boundary tokens for bigram/trigram

    # Calculate the probability using precomputed smoothed n-gram probabilities
    for i in range(n - 1, len(tokens)):  
        ngram = tuple(tokens[i - n + 1:i + 1])  
        prob_word = ngram_probs[n-1].get(ngram, 0)  
        # Apply Laplace smoothing if the n-gram is unseen
        if prob_word == 0:  
            if n!=1:
                prefix = ngram[:-1]  
                prefix_count = ngram_count[n-2].get(prefix, 0)  
            else:
                prefix_count=token_size
            prob_word = (alpha) / (prefix_count + alpha * vocabulary_size)
        prob *= prob_word

    return prob


Function to calculate perplexity of a sentence, fall back to computing again using ngram counts if the precomputation is not found. The perplexity is calculated using logarith instead of inverse to avoid underflowing.

In [89]:
def calculate_perplexity(sentence, ngram_probs,ngram_count,token_size,vocabulary_size, n, alpha=1):
    tokens = [word for sentence in split_sentences_and_tokenize(preprocess(sentence)) for word in sentence]
    offset=len(tokens)+n-1
    if n > 1:
        tokens = ['<s>'] * (n - 1) + tokens + ['</s>']  # Add boundary tokens for bigram/trigram
    total_log_prob = 0.0
    for i in range(n - 1, len(tokens)):  
        ngram = tuple(tokens[i - n + 1:i + 1])  
        prob_word = ngram_probs[n-1].get(ngram, 0)  
        
        # Apply Laplace smoothing if the n-gram is unseen
        if prob_word == 0:
            if n!=1:
                prefix = ngram[:-1]  # Prefix is the first (n-1) tokens
                prefix_count = ngram_count[n-2].get(prefix, 0)  # Get prefix count (smoothed)
            else:
                prefix_count=token_size
            prob_word = (alpha) / (prefix_count + alpha * vocabulary_size)
        # Perplexity calculation using logarith instead of inverse
        total_log_prob += math.log(prob_word) if prob_word > 0 else float('-inf')

    perplexity = math.exp(-total_log_prob / (offset)) if total_log_prob > float('-inf') else float('inf')
    return perplexity


b) Calculate the probability of a sentence and compute the Perplexity of a sentence based on 1-gram, 2-gram, and 3-gram models.

The sentence chosen is "I want to speak at ted talk." This will be the base sentence to compare below.

In [90]:
sentence = "I want to speak at ted talk."
unigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1)
bigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2)
trigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3)

print("Unigram Probability:", unigram_prob)
print("Bigram Probability:", bigram_prob)
print("Trigram Probability:", trigram_prob)

print("Unigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1))
print("Bigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2))
print("Trigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3))


Unigram Probability: 1.5121885066224436e-21
Bigram Probability: 6.671326354226927e-19
Trigram Probability: 4.339151870421607e-27
Unigram Perplexity: 400.45088554805614
Bigram Perplexity: 104.6000709925499
Trigram Perplexity: 432.77235916126153


c) Analyze the results (Provide your own examples of spelling errors and calculate the probability of two similar sentences, where one has the correct word order and the other has an incorrect word order).



This is an example of spelling errors of the previous sentence, the perplexity is much higher indicating this sentence does not fit with the model.

In [91]:
#Example of spelling errors
sentence = "I went too speak ate ted take."
unigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1)
bigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2)
trigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3)

print("Unigram Probability:", unigram_prob)
print("Bigram Probability:", bigram_prob)
print("Trigram Probability:", trigram_prob)

print("Unigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1))
print("Bigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2))
print("Trigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3))


Unigram Probability: 9.250038516821093e-26
Bigram Probability: 7.7910449546226906e-31
Trigram Probability: 2.3158151749785303e-36
Unigram Perplexity: 1346.5796822248376
Bigram Perplexity: 2215.0230728151582
Trigram Perplexity: 3660.408965940057


This is an example of wrong order of the above sentence, the perplexity is higher on all 2gram and 3gram models compared with the base sentence but is not as high as the spelling error sentence. This sentence is worse than the base sentence but is better than the spelling error one.

In [92]:
#Example of spelling errors
sentence = "I speak at want to ted talk."
unigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1)
bigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2)
trigram_prob = calculate_sentence_probability(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3)

print("Unigram Probability:", unigram_prob)
print("Bigram Probability:", bigram_prob)
print("Trigram Probability:", trigram_prob)

print("Unigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 1))
print("Bigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 2))
print("Trigram Perplexity:", calculate_perplexity(sentence, ngram_probs, ngram_counts,token_size, vocabulary_size, 3))


Unigram Probability: 1.5121885066224436e-21
Bigram Probability: 1.15249704944147e-23
Trigram Probability: 1.772522028170151e-35
Unigram Perplexity: 400.45088554805614
Bigram Perplexity: 353.75833926315215
Trigram Perplexity: 2986.3508955363095


# Home Exercise
a) Improve the model by using interpolation smoothing with the "Stupid Backoff" method (Brants et al., 2007).

In [93]:
def compute_backoff_ngram_probs(vocabulary_size,token_size, n, ngram_counts, alpha=0.4):
    probs = {}
    
    for ngram, count in ngram_counts[n-1].items():
        backoff_prob = count
        current_ngram = ngram
        current_n = n
        
        while current_n >= 1:
            prefix = current_ngram[:-1]  # Drop the first token to back off
            if current_n>1:
                prefix_count = ngram_counts[current_n-2].get(prefix, 0)
                if (prefix_count<backoff_prob):
                    print(prefix, " ",current_ngram)
            else:
                prefix_count=token_size
            
            if backoff_prob > 0:
                backoff_prob /= prefix_count if prefix_count > 0 else 1  
                break 
            else:
                backoff_prob = alpha*prefix_count  # Apply Stupid Backoff
                current_ngram = prefix
                current_n -= 1
        
        if current_n == 0 and backoff_prob == 0:  # If reached unigram level with no valid probability
            backoff_prob = alpha / vocabulary_size
        
        probs[ngram] = backoff_prob
    
    return probs


def calculate_backoff_sentence_probability(sentence, ngram_probs, ngram_count, vocabulary_size, n, beta=0.4):
    tokens = [word for sentence in split_sentences_and_tokenize(preprocess(sentence)) for word in sentence]
    prob = 1.0

    if n > 1:
        tokens = ['<s>'] * (n - 1) + tokens + ['</s>']  # Add boundary tokens for bigram/trigram

    # Calculate the probability using precomputed n-gram probabilities
    for i in range(n - 1, len(tokens)):  
        ngram = tuple(tokens[i - n + 1:i + 1])  
        prob_word = ngram_probs[n-1].get(ngram, 0)  
        currnet_n=n
        discount=1
        prefix = ngram
        while prob_word == 0:  
            currnet_n-=1
            discount*=beta
            prefix = prefix[:-1]  
            if currnet_n != 0:
                # Back off to the (n-1)-gram
                prob_word = ngram_probs[currnet_n-1].get(prefix,0)
                prob_word = discount * prob_word
            else:
                # For unigrams, use the raw frequency count divided by the token size
                prob_word = beta  / vocabulary_size
                break
                
        prob *= prob_word

    return prob

def calculate_backoff_perplexity(sentence, ngram_probs, ngram_count, vocabulary_size, n, beta=0.4):
    tokens = [word for sentence in split_sentences_and_tokenize(preprocess(sentence)) for word in sentence]
    offset=len(tokens)+n-1
    if n > 1:
        tokens = ['<s>'] * (n - 1) + tokens + ['</s>']  # Add boundary tokens for bigram/trigram

    total_log_prob = 0.0
    for i in range(n - 1, len(tokens)):  
        ngram = tuple(tokens[i - n + 1:i + 1])  
        prob_word = ngram_probs[n-1].get(ngram, 0)  
        currnet_n=n
        discount=1
        prefix = ngram
        while prob_word == 0:  
            currnet_n-=1
            discount*=beta
            prefix = prefix[:-1]  
            if currnet_n != 0:
                # Back off to the (n-1)-gram
                prob_word = ngram_probs[currnet_n-1].get(prefix,0)
                prob_word = discount * prob_word
            else:
                # For unigrams, use the raw frequency count divided by the token size
                prob_word = beta  / vocabulary_size
                break
        
        # Perplexity calculation using logarithm
        total_log_prob += math.log(prob_word) if prob_word > 0 else float('-inf')

    perplexity = math.exp(-total_log_prob / (offset)) if total_log_prob > float('-inf') else float('inf')
    return perplexity


In [62]:
backoff_ngram_probs = []
for x in range(3):
    backoff_ngram_probs.append(compute_backoff_ngram_probs(vocabulary_size,token_size,x+1,ngram_counts))
print("Sample Backoff Unigram Probabilities:", list(backoff_ngram_probs[0].items())[1:15])
print("Sample Backoff Bigram Probabilities:", list(backoff_ngram_probs[1].items())[1:15])
print("Sample Backoff Trigram Probabilities:", list(backoff_ngram_probs[2].items())[1:15])


Sample Backoff Unigram Probabilities: [(('thank',), 0.0006629988811335532), (('you',), 0.013834661734652661), (('so',), 0.007285840832537713), (('much',), 0.0011766794387009357), (('chris',), 6.355600438970347e-05), (('.',), 0.051147013588860805), (('and',), 0.030703293132671627), (('it',), 0.014779578715579096), (("'s",), 0.01089181453541135), (('truly',), 8.869763664828095e-05), (('a',), 0.021803793425629072), (('great',), 0.0006069726041715456), (('honor',), 3.535143216053787e-05), (('to',), 0.026919030799010008)]
Sample Backoff Bigram Probabilities: [(('<s>', 'thank'), 0.007305741423916667), (('thank', 'you'), 0.9626564003849856), (('you', 'so'), 0.0032840419545584532), (('so', 'much'), 0.030671407801853248), (('much', 'chris'), 0.0004338394793926247), (('chris', '.'), 0.11244979919678715), (('.', '</s>'), 0.9994360827211753), (('<s>', 'and'), 0.1362967464281908), (('and', 'it'), 0.03724348342955952), (('it', "'s"), 0.28412790245840064), (("'s", 'truly'), 0.0002929321334833146), ((

b) Compare with the results from In Class Exercise.

With backoff calculation, the probability and perplexity is much smaller for both bigram and trigram as it does not need to compensate for the vocabulary size in Laplace smoothing.

In [94]:
sentence = "I want to speak at ted talk."
unigram_prob = calculate_backoff_sentence_probability(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 1)
bigram_prob = calculate_backoff_sentence_probability(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 2)
trigram_prob = calculate_backoff_sentence_probability(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 3)

print("Unigram Probability:", unigram_prob)
print("Bigram Probability:", bigram_prob)
print("Trigram Probability:", trigram_prob)

print("Unigram Perplexity:", calculate_backoff_perplexity(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 1))
print("Bigram Perplexity:", calculate_backoff_perplexity(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 2))
print("Trigram Perplexity:", calculate_backoff_perplexity(sentence, backoff_ngram_probs, ngram_counts, vocabulary_size, 3))


Unigram Probability: 1.6202060302947543e-21
Bigram Probability: 2.5918747427400836e-12
Trigram Probability: 2.670898297559592e-11
Unigram Perplexity: 397.0120782648672
Bigram Perplexity: 19.381001767329447
Trigram Perplexity: 11.411277452192229


c) Use the newly built model to generate the next words for a given word sequence.

In [ ]:
def generate_next_word(sequence, backoff_ngram_probs,num_words, max_n=3 ):
    # Base case: if num_words is 0, return the current sequence
    if num_words == 0:
        return sequence

    # Start with the highest available n-gram and back off
    for n in range(min(max_n, len(backoff_ngram_probs)), 0, -1):
        if len(sequence) >= n - 1:
            ngram_key = tuple(sequence[-(n-1):]) 
            ngram_candidates = {}
            for ngram, prob in backoff_ngram_probs[n-1].items():
                if ngram[:-1] == ngram_key: 
                    ngram_candidates[ngram[-1]] = prob  
            if ngram_candidates:
                next_word = max(ngram_candidates, key=ngram_candidates.get)
                return generate_next_word(sequence + [next_word], backoff_ngram_probs,num_words - 1,max_n)

    # If no match found, fall back to unigram sampling
    unigram_probs = {k[0]: v for k, v in backoff_ngram_probs[0].items()}
    next_word = max(unigram_probs, key=unigram_probs.get)
    return generate_next_word(sequence + [next_word], backoff_ngram_probs, max_n, num_words - 1)


In [148]:
current_sequence = ["ted","talk","is"]
next_word = generate_next_word(current_sequence, backoff_ngram_probs,5)
print(next_word)


['ted', 'talk', 'is', 'about', 'the', 'future', '.', '</s>']


d) Combine with a function that calculates the distance between words to predict the correct word for a misspelled word position. (from difflib import get_close_matches)



In [118]:
from difflib import get_close_matches
def get_correct_word(word, vocabulary):
    # Use difflib to find close matches for a potentially misspelled word
    close_matches = get_close_matches(word, vocabulary, n=1, cutoff=0.8)
    
    if close_matches:
        return close_matches[0]
    else:
        return word  


In [119]:
vocabulary=set(all_tokens)

In [150]:
current_sequence = ["ted","tlk","is"]
corrected_sequence = [get_correct_word(word, vocabulary) for word in current_sequence]
print(corrected_sequence)
next_word = generate_next_word(corrected_sequence, backoff_ngram_probs,5)
print(next_word)


['ted', 'talk', 'is']
['ted', 'talk', 'is', 'about', 'the', 'future', '.', '</s>']
